In [33]:
import pandas as pd
from json import load, dump
from dataset import process_BindingDB, get_cliffs, split_data

In [17]:
df = pd.read_csv('/mnt/data/BindingDB_All.tsv', sep = '\t', on_bad_lines='skip')
df.head()

,BindingDB Reactant_set_id,Ligand SMILES,Ligand InChI,Ligand InChI Key,BindingDB MonomerID,BindingDB Ligand Name,Target Name,Target Source Organism According to Curator or DataSource,Ki (nM),IC50 (nM),...,Unnamed: 233,Unnamed: 234,Unnamed: 235,Unnamed: 236,Unnamed: 237,Unnamed: 238,Unnamed: 239,Unnamed: 240,Unnamed: 241,Unnamed: 242
0,2,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(C\C=C\c2cn...,InChI=1S/C31H34N6O3/c38-29-27(17-23-9-3-1-4-10...,UZLMEAPBHYEHAC-UNTBESQGSA-N,22,"(4R,5S,6S,7R)-4,7-dibenzyl-5,6-dihydroxy-1,3-b...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.25,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CC2CC2)C(=...,InChI=1S/C29H34N4O3/c34-27-25(16-21-8-3-1-4-9-...,HYNYUFZPPJMPOB-UTWJFGBXSA-N,23,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.41,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,OCCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@...,InChI=1S/C29H40N2O4/c32-18-10-2-1-9-17-30-25(1...,YXVAZXDWVZTGGD-VIJSPRBVSA-N,24,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,OCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H...,InChI=1S/C28H38N2O4/c31-17-9-3-8-16-29-24(18-2...,WWTSWTPNILRSJX-XDZXDJIYSA-N,25,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,0.99,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,CCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H](...,InChI=1S/C27H36N2O3/c1-2-3-16-28-23(17-20-10-6...,QDUGHFLRTUTUJH-XPGKHFPBSA-N,26,"(4R,5S,6S,7R)-4,7-dibenzyl-1-butyl-3-(cyclopro...",Dimer of Gag-Pol polyprotein [501-599],Human immunodeficiency virus 1,1.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
X_drug, X_SMILES, X_target, y = process_BindingDB('/mnt/data/BindingDB_All.tsv',
                                         y = 'Ki', 
                                         binary = False, 
                                         convert_to_log = True)

In [74]:
bindingdb_d_t_ki = pd.DataFrame({'drug': X_drug, 'SMILES': X_SMILES, 'target': X_target, 'Ki': y})

In [67]:
bindingdb_d_t_ki.to_csv('../analysis/bindingdb_d_t_ki.csv', index=False)

In [2]:
data_ki= pd.read_csv('../analysis/bindingdb_d_t_ki.csv')

In [3]:
data_ki.head()

,drug,SMILES,target,Ki
0,"(4R,5S,6S,7R)-4,7-dibenzyl-5,6-dihydroxy-1,3-b...",O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(C\C=C\c2cn...,Dimer of Gag-Pol polyprotein [501-599],9.602060
1,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CC2CC2)C(=...,Dimer of Gag-Pol polyprotein [501-599],9.387216
2,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",OCCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@...,Dimer of Gag-Pol polyprotein [501-599],9.096910
3,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",OCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H...,Dimer of Gag-Pol polyprotein [501-599],9.004365
4,"(4R,5S,6S,7R)-4,7-dibenzyl-1-butyl-3-(cyclopro...",CCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H](...,Dimer of Gag-Pol polyprotein [501-599],8.958607


In [5]:
data_ki.columns = 'drug', 'SMILES', 'target', 'affinity'

In [6]:
data_ki.head()

,drug,SMILES,target,affinity
0,"(4R,5S,6S,7R)-4,7-dibenzyl-5,6-dihydroxy-1,3-b...",O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(C\C=C\c2cn...,Dimer of Gag-Pol polyprotein [501-599],9.602060
1,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CC2CC2)C(=...,Dimer of Gag-Pol polyprotein [501-599],9.387216
2,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",OCCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@...,Dimer of Gag-Pol polyprotein [501-599],9.096910
3,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",OCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H...,Dimer of Gag-Pol polyprotein [501-599],9.004365
4,"(4R,5S,6S,7R)-4,7-dibenzyl-1-butyl-3-(cyclopro...",CCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H](...,Dimer of Gag-Pol polyprotein [501-599],8.958607


In [ ]:
cliff_pairs = get_cliffs(data_ki, threshold_affinity=1, threshold_similarity=0.9)

In [ ]:
cliff_pairs

In [16]:
cliff_pairs_no_dup = cliff_pairs.drop_duplicates()

In [17]:
cliff_pairs.shape

(498786, 6)

In [18]:
cliff_pairs_no_dup.shape

(252803, 6)

In [ ]:
#cliff_pairs_no_dup.to_csv('../analysis/bindindb_ki_cliff_pairs_ta1_ts0.9.csv', index=False)

# Split data for DDC task

In [16]:
cliff_pairs_no_dup =  pd.read_csv('../analysis/bindindb_ki_cliff_pairs_ta1_ts0.9.csv')

In [17]:
target_ids = {t:i for i, t in enumerate(set(cliff_pairs_no_dup.target))}
dump(target_ids, open('../analysis/target_mapping_bdb.json', 'w'))

In [18]:
target_ids = load(open('../analysis/target_mapping_bdb.json'))
cliff_pairs_no_dup.loc[:, 'target'] = cliff_pairs_no_dup['target'].apply(target_ids.__getitem__)

In [22]:
cliff_pairs_split_random = split_data(cliff_pairs_no_dup, split='random')
cliff_pairs_split_random.to_csv('../analysis/bindingdb_ki_cliff_pairs_ta1_ts0.9_r_wt.csv', index=False)

In [23]:
cliff_pairs_split_random['cliff'].value_counts()

cliff
0    28678
1    10242
Name: count, dtype: int64

In [24]:
cliff_pairs_split_cb = split_data(cliff_pairs_no_dup, split='compound-based')
cliff_pairs_split_cb.to_csv('../analysis/bindingdb_ki_cliff_pairs_ta1_ts0.9_cb_wt.csv', index=False)

In [25]:
cliff_pairs_split_cb['cliff'].value_counts()

cliff
0    28678
1    10242
Name: count, dtype: int64

In [26]:
cliff_pairs_split_random.head()

,drug1,drug2,smiles1,smiles2,cliff,target,split
16104,3-chloro-N-{4-chloro-2-[(4-chlorophenyl)carbam...,3-chloro-N-{4-chloro-2-[(4-chlorophenyl)carbam...,CN1CCN(Cc2csc(C(=O)Nc3ccc(Cl)cc3C(=O)Nc3ccc(Cl...,Clc1c(Cn2nccn2)csc1C(=O)Nc1ccc(Cl)cc1C(=O)Nc1c...,0,120,0
14061,5-chloro-N-[(3S)-1-[(2S)-1-(morpholin-4-yl)-1-...,5-chloro-N-[(3S)-1-[(2S)-1-(morpholin-4-yl)-1-...,C[C@H](N1CC[C@H](NS(=O)(=O)c2cc3cc(Cl)ccc3o2)C...,C[C@H](N1CC[C@H](NS(=O)(=O)c2cc3cc(Cl)ccc3[nH]...,0,120,0
20357,"Azabicyclooctane scaffold, 14i::N-[(3aR,6S,6aS...","Azabicyclooctane scaffold, 14k::N-[(3aR,6S,6aS...",CN[C@@H](C)C(=O)N[C@H](C(=O)N1CC[C@H]2CC[C@H](...,CN[C@@H](C)C(=O)N[C@H](C(=O)N1CC[C@H]2CC[C@H](...,0,19,0
12027,"(3R)-2-[(4-chloro-1,1-biphenyl-4-yl)methyl]-N-...","(3R)-N-hydroxy-1,1-dioxo-2-{[4-(pyridin-4-yl)p...",ONC(=O)[C@H]1CCCS(=O)(=O)N1Cc1ccc(cc1)-c1ccc(C...,ONC(=O)[C@H]1CCCS(=O)(=O)N1Cc1ccc(cc1)-c1ccncc1,0,252,0
6407,"iodinated benzimidazole, 12a","iodinated benzimidazole, 13a",Clc1nc2c(I)c(I)c(I)c(I)c2[nH]1,Brc1nc2c(I)c(I)c(I)c(I)c2[nH]1,0,16,0


In [27]:
cliff_pairs_split_cb.head()

,drug1,drug2,smiles1,smiles2,cliff,target,split
0,"(4R,5S,6S,7R)-4,7-dibenzyl-5,6-dihydroxy-1,3-b...","(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(C\C=C\c2cn...,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CC2CC2)C(=...,0,98,0
1,"(4R,5S,6S,7R)-4,7-dibenzyl-5,6-dihydroxy-1,3-b...","(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(C\C=C\c2cn...,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CC2CC2)C(=...,1,98,0
2,"(4R,5S,6S,7R)-4,7-dibenzyl-5,6-dihydroxy-1,3-b...","(4R,5S,6S,7R)-4,7-dibenzyl-5,6-dihydroxy-1,3-b...",O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(C\C=C\c2cn...,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(C\C=C\c2cc...,1,98,0
3,"(4R,5S,6S,7R)-4,7-dibenzyl-5,6-dihydroxy-1,3-b...","(4R,5S,6S,7R)-4,7-dibenzyl-5,6-dihydroxy-1,3-b...",O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(C\C=C\c2cn...,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CCCc2ccccc...,1,98,0
5,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...","(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CC2CC2)C(=...,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CC2CC2)C(=...,1,98,0


In [35]:
cliff_pairs_split_random = pd.read_csv('../analysis/bindingdb_ki_cliff_pairs_ta1_ts0.9_r_wt.csv')

In [34]:
cliff_pairs_split_random['target'].nunique()

676

In [36]:
cliff_pairs_split_random

,drug1,drug2,smiles1,smiles2,cliff,target,split
0,"Azabicyclooctane scaffold, 14l::N-[(3aR,6S,6aS...","Azabicyclooctane scaffold, 14p::N-[(3aR,6S,6aS...",CN[C@@H](C)C(=O)N[C@H](C(=O)N1CC[C@H]2CC[C@H](...,CN[C@@H](C)C(=O)N[C@H](C(=O)N1CC[C@H]2CC[C@H](...,0,411,0
1,"US8952157, 140::US9303025, 140","US8952157, 133::US9303025, 133",CC1(C)CCC(CN2CCN(CC2)c2ccc(C(=O)NS(=O)(=O)c3cc...,CC1(C)CCC(CN2CCN(CC2)c2ccc(C(=O)NS(=O)(=O)c3cc...,0,530,0
2,"US8623889, 33","US8623889, 46",O=C(Nc1cc2ccc(cc2cn1)N1CCCC1)C1CC1,O=C(Nc1cc2ccc(cc2cn1)N1CCNCC1)C1CC1,1,139,0
3,2-(hydrazinecarbonyl)-3-(2-methylphenyl)-1H-in...,3-(2-fluorophenyl)-2-(hydrazinecarbonyl)-1H-in...,Cc1ccccc1-c1c([nH]c2ccc(cc12)S(N)(=O)=O)C(=O)NN,NNC(=O)c1[nH]c2ccc(cc2c1-c1ccccc1F)S(N)(=O)=O,0,226,0
4,VIP [D- Arg14]::VIP [D-Arg14],VIP [D- Asn24]::VIP [D-Asn24],CC[C@H](C)[C@H](NC(=O)[C@H](CO)NC(=O)[C@H](CC(...,CC[C@H](C)[C@H](NC(=O)[C@H](CO)NC(=O)[C@@H](CC...,1,65,0
...,...,...,...,...,...,...,...
252798,"(phenylmethyl) N-[[(6aR,9S,10aR)-4,7-dimethyl-...","(phenylmethyl) N-[[(6aR,9S,10aR)-4,7-dimethyl-...",CN1C[C@H](CNC(=O)OCc2ccccc2)C[C@H]2[C@H]1Cc1cn...,CN1C[C@H](CNC(=O)OCc2ccccc2)C[C@H]2[C@H]1Cc1cn...,0,349,2
252799,"US9062078, 69::US9475819, 69::US9637496, 69::U...","US9062078, 177::US9475819, 177::US9637496, 177...",Cc1cc(on1)-c1ccc(C)nc1C(=O)N1C2CCC1C(COc1ccc(F...,Cc1coc(n1)-c1ccc(C)nc1C(=O)N1C2CCC1C(COc1ccc(F...,0,176,2
252800,"US9120756, 12","US9120756, 13",CC(C(=O)NCc1cc(nn1-c1ccc(C)cc1)C(C)(C)C)c1ccc(...,CC(C(=O)NCc1cc(nn1-c1ccc(cc1)C(C)(C)C)C(C)(C)C...,0,508,2
252801,"US9067949, 189","US9067949, 209",COc1cc(cc2c3CNCCc3oc12)S(=O)(=O)c1ccc(Cl)cc1,Fc1ccc(cc1)S(=O)(=O)c1cc(C#N)c2oc3CCNCc3c2c1,1,123,2


# Split data for DTI task

In [28]:
data_ki= pd.read_csv('../analysis/bindingdb_d_t_ki.csv')

In [31]:
data_ki.head()

,drug,SMILES,target,affinity
0,"(4R,5S,6S,7R)-4,7-dibenzyl-5,6-dihydroxy-1,3-b...",O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(C\C=C\c2cn...,Dimer of Gag-Pol polyprotein [501-599],9.602060
1,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CC2CC2)C(=...,Dimer of Gag-Pol polyprotein [501-599],9.387216
2,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",OCCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@...,Dimer of Gag-Pol polyprotein [501-599],9.096910
3,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",OCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H...,Dimer of Gag-Pol polyprotein [501-599],9.004365
4,"(4R,5S,6S,7R)-4,7-dibenzyl-1-butyl-3-(cyclopro...",CCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H](...,Dimer of Gag-Pol polyprotein [501-599],8.958607


In [34]:
with open('../analysis/target_mapping_bdb.json', 'r') as f:
    target_ids = load(f)

# Get the highest current ID to ensure new IDs are unique
current_max_id = max(target_ids.values())

# Function to get or create target ID
def get_or_create_target_id(target):
    global current_max_id
    if target in target_ids:
        return target_ids[target]
    else:
        current_max_id += 1
        target_ids[target] = current_max_id
        return current_max_id

In [35]:
data_ki['target'] = data_ki['target'].apply(get_or_create_target_id)


In [36]:
data_ki.head()

,drug,SMILES,target,affinity
0,"(4R,5S,6S,7R)-4,7-dibenzyl-5,6-dihydroxy-1,3-b...",O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(C\C=C\c2cn...,529,9.602060
1,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CC2CC2)C(=...,529,9.387216
2,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",OCCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@...,529,9.096910
3,"(4R,5S,6S,7R)-4,7-dibenzyl-1-(cyclopropylmethy...",OCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H...,529,9.004365
4,"(4R,5S,6S,7R)-4,7-dibenzyl-1-butyl-3-(cyclopro...",CCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H](...,529,8.958607


In [37]:
data_ki['target'].nunique()

1018

In [38]:
data_ki_aff_split = split_data(data_ki, 'random')

In [43]:
data_ki_aff_split = data_ki_aff_split.rename(columns={'SMILES': 'smiles'})

In [46]:
data_ki_aff_split

,drug,smiles,target,affinity,split
0,(risperidone)3-{2-[4-(6-Fluoro-benzo[d]isoxazo...,Cc1nc2CCCCn2c(=O)c1CCN1CCC(CC1)c1noc2cc(F)ccc12,620,9.522879,0
1,4-Amino-3-fluorobenzenesulfonamide::4-amino-3-...,Nc1ccc(cc1F)S(N)(=O)=O,182,7.221849,0
2,5-(4-chloro-3-methylphenyl)-1-[(4-methylphenyl...,Cc1ccc(Cn2nc(cc2-c2ccc(Cl)c(C)c2)C(=O)NC2[C@@]...,821,5.000000,0
3,"2-{[(2-chlorophenyl)methyl](3,5-dichloro-2-hyd...",CC(N(Cc1ccccc1Cl)S(=O)(=O)c1cc(Cl)cc(Cl)c1O)C(...,89,5.619789,0
4,AcNH-4-NO2-Phe-c[D-Cys-Tyr-D-Trp-Lys-Thr-Cys]-...,CC(C)[C@H]1NC(=O)[C@H](CCCCN)NC(=O)[C@@H](Cc2c...,332,7.000000,0
...,...,...,...,...,...
78623,"US9079866, 302::US9745328, Compound 302::US988...",Nc1ncc(cn1)-c1ccc(cc1F)-c1ccccc1S(N)(=O)=O,517,5.602060,2
78624,2-aminobenzene-1-sulfonamide::CHEMBL6705::US10...,Nc1ccccc1S(N)(=O)=O,536,5.204815,2
78625,CHEMBL2016866,NC(=N)c1ccc(CNC(=O)[C@@H]2CCC(=O)NCc3cccc(CNC(...,106,5.464706,2
78626,"US8604061, 260",NC1=NC(CO1)c1cc(Cl)ccc1Cl,397,8.886057,2


In [45]:
data_ki_aff_split.to_csv('../analysis/bindingdb_ki_d_t_aff_smiles_split.csv', index=False)

In [53]:
data_ki_aff_split.split.value_counts()

split
0    55039
2    15726
1     7863
Name: count, dtype: int64